In [1]:
import numpy as np
import pandas as pd
import geopandas as gpd
import plotly.express as px

# Exploration du fichier des vitesses

Ce fichier contient les vitesses maximales autorisées sur les tronçons du réseau ferré national.

In [2]:
speeds = gpd.read_file("../data/vitesse-maximale-nominale-sur-ligne.geojson")
print(speeds.shape)
speeds.head()

(2529, 19)


,code_ligne,lib_ligne,v_max,rg_troncon,pkd,pkf,idgaia,x_d_l93,y_d_l93,x_f_l93,y_f_l93,x_d_wgs84,y_d_wgs84,x_f_wgs84,y_f_wgs84,c_geo_d,c_geo_f,geo_point_2d,geometry
0,242000,Ligne de Creil à Jeumont,140,1,222+152,224+841,77986fb2-e28c-11e8-92ff-01b064e0362d,7.641287e+05,7.016495e+06,7.658409e+05,7.018610e+06,3.897931,50.243157,3.922249,50.261966,"{ ""lon"": 3.897931336245875, ""lat"": 50.24315723...","{ ""lon"": 3.9222490335686517, ""lat"": 50.2619658...","{ ""lon"": 3.9108305177413909, ""lat"": 50.2517217...","LINESTRING (3.89793 50.24316, 3.8982 50.24339,..."
1,366000,Ligne de Mantes-la-Jolie à Cherbourg,200,1,204+738,238+908,77a4fa62-e28c-11e8-92ff-01b064e0362d,4.856911e+05,6.893278e+06,4.559102e+05,6.902591e+06,0.064633,49.103467,-0.348220,49.176484,"{ ""lon"": 0.064633482429558339, ""lat"": 49.10346...","{ ""lon"": -0.34821966895941142, ""lat"": 49.17648...","{ ""lon"": -0.14445964061185351, ""lat"": 49.10945...","LINESTRING (0.06463 49.10347, 0.06453 49.10343..."
2,995000,Ligne de Bastia à Ajaccio (ligne centrale),60,1,091+100,092+550,77909e06-e28c-11e8-92ff-01b064e0362d,1.209888e+06,6.140752e+06,1.210450e+06,6.139784e+06,9.167456,42.187707,9.173311,42.178645,"{ ""lon"": 9.1674555515289811, ""lat"": 42.1877074...","{ ""lon"": 9.1733106648204128, ""lat"": 42.1786453...","{ ""lon"": 9.1708395291329747, ""lat"": 42.1833669...","LINESTRING (9.16746 42.18771, 9.1676 42.18756,..."
3,995000,Ligne de Bastia à Ajaccio (ligne centrale),70,1,028+280,031+800,778e2f14-e28c-11e8-92ff-01b064e0362d,1.224691e+06,6.177971e+06,1.221449e+06,6.177839e+06,9.382049,42.510609,9.342656,42.511779,"{ ""lon"": 9.3820493826610285, ""lat"": 42.5106094...","{ ""lon"": 9.3426559493521957, ""lat"": 42.5117794...","{ ""lon"": 9.3623621168976765, ""lat"": 42.5093457...","LINESTRING (9.38205 42.51061, 9.3752 42.50851,..."
4,890306,Raccordement de Culoz,60,1,000+000,000+606,779306f0-e28c-11e8-92ff-01b064e0362d,9.162122e+05,6.531116e+06,9.160776e+05,6.530596e+06,5.786444,45.845357,5.784475,45.840723,"{ ""lon"": 5.7864437996495486, ""lat"": 45.8453573...","{ ""lon"": 5.7844746068288542, ""lat"": 45.8407233...","{ ""lon"": 5.7845964512461716, ""lat"": 45.8432853...","LINESTRING (5.78644 45.84536, 5.78532 45.84474..."


## Réduction des données

Les colonnes ```"rg_troncon"```, ```"pkd"```, ```"pkf"```, ```"geo_point_2d"```, ```"geometry"``` sont déjà présentes dans le fichier ```"formes-des-lignes-du-rfn.geojson"```, donc elles sont redondantes. On garde néanmoins la colonne ```"code_ligne"``` comme clé pour le merging plus tard.

Les colonnes ```'x_d_l93','y_d_l93','x_f_l93','y_f_l93','x_d_wgs84','y_d_wgs84','x_f_wgs84','y_f_wgs84','c_geo_d','c_geo_f'``` sont des coordonnées pour différentes projections cartographiques. Elles ne sont donc pas pertinentes.

<https://fr.wikipedia.org/wiki/Projection_conique_conforme_de_Lambert>

<https://fr.wikipedia.org/wiki/WGS_84#:~:text=WGS%2084%20(World%20Geodetic%20System,g%C3%A9o%C3%AFde%20(EGM96%20ou%20suivant).>

```"idgaia"``` est un Id qui n'apparait nulle part ailleurs dans les données, on n'en a pas d'utilité, d'autant plus qu'on a déjà un identifiant pour les tronçons, avec ```"code_ligne"```.

In [3]:
speeds.dtypes

code_ligne        object
lib_ligne         object
v_max             object
rg_troncon         int32
pkd               object
pkf               object
idgaia            object
x_d_l93          float64
y_d_l93          float64
x_f_l93          float64
y_f_l93          float64
x_d_wgs84        float64
y_d_wgs84        float64
x_f_wgs84        float64
y_f_wgs84        float64
c_geo_d           object
c_geo_f           object
geo_point_2d      object
geometry        geometry
dtype: object

In [4]:
def process_speeds(speeds_df, ignore_na=True):
    relevant_columns = ["code_ligne", "lib_ligne", "v_max"]
    speeds_df_processed = speeds_df[relevant_columns]
    if ignore_na:
        speeds_df_processed = speeds_df_processed[~speeds_df_processed["v_max"].isna()]
    else:
        max_speed = speeds_df_processed[~speeds_df_processed["v_max"].isna()]["v_max"].astype(int).max()
        speeds_df_processed.loc[:,"v_max"] = speeds_df_processed.loc[:,"v_max"].fillna(max_speed)
    speeds_df_processed.loc[:,"v_max"] = speeds_df_processed.loc[:,"v_max"].astype(int)
    
    return speeds_df_processed

speeds_processed = process_speeds(speeds, ignore_na=True)
print(speeds_processed.shape)
speeds_processed.head()

(2165, 3)


,code_ligne,lib_ligne,v_max
0,242000,Ligne de Creil à Jeumont,140
1,366000,Ligne de Mantes-la-Jolie à Cherbourg,200
2,995000,Ligne de Bastia à Ajaccio (ligne centrale),60
3,995000,Ligne de Bastia à Ajaccio (ligne centrale),70
4,890306,Raccordement de Culoz,60


## Bonus: Histogramme générique

In [5]:
def generate_histogram(speeds_df):
    fig = px.histogram(speeds_df, x="v_max", nbins=50)
    return fig

generate_histogram(speeds_processed)

In [6]:
generate_histogram(process_speeds(speeds, ignore_na=False))

Si suppose que lorsque "v_max" vaut ```NA```, alors c'est qu'il n'y a pas de limitations de vitesse et que donc la vitesse maximale est de 350, On a une anomalie dans l'histogramme.

De plus, en analysant les lignes où la limitation de vitesse est haute, il est spécifié s'agit de LGV (lignes à grande vitesse). Or, dans les lignes où "v_max" vaut ```NA```, cette précision n'est pas présente. On ne peut donc pas tirer de conclusions sur les limitations de vitesses qui ne sont pas présentes dans le dataset.

In [7]:
speeds[["code_ligne", "lib_ligne", "v_max"]][speeds["v_max"].isna()].head()

,code_ligne,lib_ligne,v_max
15,912000,Ligne de Livron à Aspres-sur-Buëch,None
24,314611,Voie-mère de Calais,None
30,990000,Ligne de la grande ceinture de Paris,None
49,985000,Ligne de Choisy-le-Roi à Massy-Verrières,None
52,830000,Ligne de Paris-Lyon à Marseille-St-Charles,None


In [8]:
speeds_processed.query("v_max >= 300").head()

,code_ligne,lib_ligne,v_max
82,005000,Ligne de Paris à Strasbourg (LGV),320
242,005000,Ligne de Paris à Strasbourg (LGV),300
275,005000,Ligne de Paris à Strasbourg (LGV),320
414,837000,Ligne de Perpignan à Figueras (LGV),350
764,408000,Ligne de Connerré à Rennes (LGV),300


On doit donc légerement modifier ```process_speeds``` de sorte que les données qui ne sont pas renseignées soient prises en compte correctement.

## ```process_speeds```, deuxième version

In [9]:
def process_speeds(speeds_df):
    relevant_columns = ["code_ligne", "lib_ligne", "v_max"]
    speeds_df_processed = speeds_df[relevant_columns]
    speeds_df_processed.loc[:,"v_max"] = pd.to_numeric(speeds_df_processed["v_max"], errors="coerce")
    
    return speeds_df_processed

speeds_processed = process_speeds(speeds)
speeds_processed.head()

,code_ligne,lib_ligne,v_max
0,242000,Ligne de Creil à Jeumont,140.0
1,366000,Ligne de Mantes-la-Jolie à Cherbourg,200.0
2,995000,Ligne de Bastia à Ajaccio (ligne centrale),60.0
3,995000,Ligne de Bastia à Ajaccio (ligne centrale),70.0
4,890306,Raccordement de Culoz,60.0


In [10]:
generate_histogram(speeds_processed)